In [9]:
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import import_ipynb
import feature_engineering

import surprise

In [10]:
ratings = pickle.load(open("data/ratings_feature_set.pkl", "rb"))
ratings_train, ratings_test = feature_engineering.train_test_split(ratings, size = 0.1)

In [19]:
# def baseline_model():
bsl_options = {'method':'als', 'n_epochs':20,}
model = surprise.BaselineOnly(bsl_options = bsl_options, verbose = True)

In [17]:
results

In [ ]:
ratings_train, ratings_test = feature_engineering.train_test_split(ratings, size = 0.1)